In [2]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

importation du tableau principal

In [33]:
df = pd.read_csv(r"tableau_gbif.csv")


D:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2,12,14,15,20,21,22,23,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Selection des colonnes

In [35]:
newtab = df[['id','identifier_y','kingdom','phylum','class','order','family','genus','species','verbatimScientificName','countryCode','decimalLatitude','decimalLongitude','hasCoordinate']]

In [14]:
newtab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2503147 entries, 0 to 2503146
Data columns (total 14 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   id                      int64  
 1   identifier_y            object 
 2   kingdom                 object 
 3   phylum                  object 
 4   class                   object 
 5   order                   object 
 6   family                  object 
 7   genus                   object 
 8   species                 object 
 9   verbatimScientificName  object 
 10  countryCode             object 
 11  decimalLatitude         float64
 12  decimalLongitude        object 
 13  hasCoordinate           object 
dtypes: float64(1), int64(1), object(12)
memory usage: 267.4+ MB


la colonne decimalLongitude est en format object, cela pose problème lors de la lecture des coordonnées. Utilisation de coerce pour générer un NaN si ce n'est pas possible, préférable à la donnée d'origine

In [24]:
newtab.decimalLongitude = pd.to_numeric(newtab.decimalLongitude,errors = 'coerce')

In [25]:
newtab["class"].unique()

array(['Agaricomycetes', nan, 'Coriolopsis', 'Earliella'], dtype=object)

Trie par classe, en effet après analyse, d'autres classes 'parasitent' le tableau

In [26]:
newtab = newtab[newtab['class']=='Agaricomycetes']

In [27]:
newtab.hasCoordinate.unique()

array([True, False, 'True', 'true', 'false', 'False'], dtype=object)

Harmonisation de la colonne hasCoordinate qui dispose de string et de bool, de majuscules et minuscules

In [28]:
newtab.hasCoordinate = newtab.hasCoordinate.replace (['True','true'],True)
newtab.hasCoordinate = newtab.hasCoordinate.replace (['False','false'],False)

In [29]:
newtab.hasCoordinate.value_counts()

True     2340729
False     162391
Name: hasCoordinate, dtype: int64

nettoyage des colonnes qui nous interressent pour les classes. Avoir des NaN dans ces colonnes n'aurait pas de sens pour l'entrainement.

In [36]:
newtab.isna().sum()

id                             0
identifier_y                 113
kingdom                       21
phylum                        21
class                         23
order                        959
family                     36431
genus                      19359
species                   136539
verbatimScientificName        44
countryCode                17320
decimalLatitude           162418
decimalLongitude          162415
hasCoordinate                 25
dtype: int64

In [30]:
newtab = newtab.dropna (subset=['genus','family','species','identifier_y'])

In [31]:
newtab.isna().sum()

id                             0
identifier_y                   0
kingdom                        0
phylum                         0
class                          0
order                          0
family                         0
genus                          0
species                        0
verbatimScientificName         0
countryCode                16229
decimalLatitude           148099
decimalLongitude          148099
hasCoordinate                  0
dtype: int64

In [32]:
newtab.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2339816 entries, 0 to 2503146
Data columns (total 14 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   id                      int64  
 1   identifier_y            object 
 2   kingdom                 object 
 3   phylum                  object 
 4   class                   object 
 5   order                   object 
 6   family                  object 
 7   genus                   object 
 8   species                 object 
 9   verbatimScientificName  object 
 10  countryCode             object 
 11  decimalLatitude         float64
 12  decimalLongitude        float64
 13  hasCoordinate           bool   
dtypes: bool(1), float64(2), int64(1), object(10)
memory usage: 252.2+ MB


In [ ]:
enregistrement du tableau

In [67]:
newtab.to_csv('premier_tableau.csv')